In [12]:
import os
import json
import pandas as pd
import numpy as np
import gensim.models
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import FunctionTransformer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import string
import re

In [13]:
path_reddit = r'C:\Users\henon\code\Stock_market\STOCK_PREDICT\data\RedditNews.csv'
path_news = r'C:\Users\henon\code\Stock_market\STOCK_PREDICT\data\Combined_News_DJIA.csv'
path_djia = r'C:\Users\henon\code\Stock_market\STOCK_PREDICT\data\upload_DJIA_table.csv'

In [14]:
reddit = pd.read_csv(path_reddit)
news = pd.read_csv(path_news)
djia = pd.read_csv(path_djia)

In [15]:
#merge dataset
news['Date'] = pd.to_datetime(news['Date'])
djia['Date'] = pd.to_datetime(djia['Date'])

df = news.merge(djia)

In [16]:
# get percentage change
df['change'] = df['Open'].pct_change()

In [17]:
df['change'] = df['change'].shift(-1)

In [18]:
# Get categorical data
def categorical(x):
    if x > 0:
        x = 1
    else:
        x = 0
    return x

In [19]:
df['target'] = df['change'].apply(categorical)

In [20]:
# Combine the top 25 daily news into 1 column
cols = df.columns[2:]
df['combined'] = df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [21]:
def clean(text):
    punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    for punctuation in punctuation:
        review1 = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    without_b=text.replace(" b ","")
    without_b=text.replace("b'","")
    without_b=text.replace('b"',"")
    tokenized = word_tokenize(without_b) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return " ".join(lemmatized)
df['cleaned'] = df['combined'].apply(clean)

In [22]:
df['cleaned']

0       Georgia two Russian warplane country move brin...
1       wont America Nato help u If wont help u help I...
2       adorable sang opening ceremony That fake Georg...
3       b refuse Israel weapon attack Iran president o...
4       expert admit legalise drug South Osetia pictur...
                              ...                        
1984    Barclays RBS share suspended trading tanking s...
1985    Scientists To Australia If You Want To Save Th...
1986    Explosion At Airport In former president Terro...
1987    Jamaica proposes marijuana dispenser tourist a...
1988    A woman Mexico City finally received birth cer...
Name: cleaned, Length: 1989, dtype: object